In [1]:
import sys
sys.path.append('C:/Projects/regionintelligence/map/')


In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import langchain as lc
import openai 
import os
import re
import json
import time
import datetime    
from typing import Optional, List
from src.paths import PROCESSED_DATA_DIR
from scripts.parallel_scraper import run_all_scrapers
import pandas as pd
import os
from dotenv import load_dotenv

# Scrape the data
def scrape_all_data():
    run_all_scrapers()
scrape_all_data()

2023-11-02 20:23:48,035 - map data - INFO - Running all scrapers in parallel
2023-11-02 20:23:48,036 - map data - INFO - Running Anaheim scraper
2023-11-02 20:23:48,039 - map data - INFO - Running Orange scraper
2023-11-02 20:23:48,039 - map data - INFO - Running Santa Ana scraper
2023-11-02 20:23:48,042 - map data - INFO - Running Fullerton scraper
2023-11-02 20:23:48,043 - map data - INFO - Running Garden Grove scraper
C:\Projects/regionintelligence/map\src\data.py:760: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['owner'], inplace=True)
2023-11-02 20:23:48,575 - map data - INFO - Anaheim scraper finished


Current Projects | City of Garden Grove
Current Projects | City of Orange, CA
['Mixed Use Building - Northeast corner of Brookhurst Street and Garden Grove Boulevard', 'Lampson Small Lot Subdivision', 'Industrial Building on Lincoln Way', 'Target Shopping Center Expansion', 'Harbor Place Pad Building', 'Choisser Apartments', 'Site C Hotel', 'Cottage Industries', 'West Grove Center Redevelopment Project', 'Brookhurst Place']
Major planning projects and monthly development reports - City of Santa Ana
Development Activity | Fullerton, CA


2023-11-02 20:24:05,389 - map data - INFO - Fullerton scraper finished
2023-11-02 20:24:22,818 - map data - INFO - Orange scraper finished


https://storage.googleapis.com/proudcity/santaanaca/uploads/2023/12/November-2023-DP-List.pdf is not available
File for October 2023 already exists in local storage.
File for September 2023 already exists in local storage.
File for August 2023 already exists in local storage.
File for July 2023 already exists in local storage.
File for June 2023 already exists in local storage.
File for May 2023 already exists in local storage.


2023-11-02 20:26:14,655 - map data - INFO - Concatenating and saving all Santa Ana data
2023-11-02 20:26:14,773 - map data - INFO - Santa Ana scraper finished
2023-11-02 20:26:16,803 - map data - INFO - All scrapers finished


In [3]:
import pandas as pd
from pathlib import Path
import re  # Importing the 're' module for regex operations
from src.paths import PROCESSED_DATA_DIR, FINAL_DATA_DIR

def load_dataframes_from_directory(base_dir_path):
    """
    Load the latest Excel files from subdirectories of the given directory into dataframes.
    """
    base_dir = Path(base_dir_path)
    dataframes = {}

    for subdir in base_dir.iterdir():
        if subdir.is_dir():
            latest_file = None
            latest_mod_time = 0
            
            # Find the latest file based on modification time
            for file in subdir.iterdir():
                if file.suffix == '.xlsx':
                    mod_time = file.stat().st_mtime
                    if mod_time > latest_mod_time:
                        latest_mod_time = mod_time
                        latest_file = file

            # Load the latest file
            if latest_file:
                try:
                    df = pd.read_excel(latest_file, engine='openpyxl')
                    if subdir.name not in dataframes:
                        dataframes[subdir.name] = {}
                    dataframes[subdir.name][latest_file.stem] = df
                except Exception as e:
                    print(f"Error reading {latest_file}: {e}")
    
    return dataframes

def clean_dataframe(df):
    """
    Clean a dataframe.
    """
    if not isinstance(df, pd.DataFrame):
        raise ValueError("Expected a DataFrame but received a Series.")

    # Drop "Unnamed: 0" column
    if "Unnamed: 0" in df.columns:
        df = df.drop(columns=["Unnamed: 0"])

    # Replace every '\n' with space
    df = df.replace('\n', '', regex=True)

    # drop applicant name column
    if "applicantName" in df.columns:
        df = df.drop(columns=["applicantName"])
    
    # drop codes from projectName column
   # if 'projectName' in df.columns:
#    df['projectName'] = df['projectName'].str.replace(r'\[.*?\] ', '', regex=True)

    return df

def concatenate_dataframes(dfs_dict):
    """
    Concatenate all dataframes in the dictionary into a single dataframe.
    """
    dfs_list = [df for _, file_dict in dfs_dict.items() for _, df in file_dict.items()]
    concatenated_df = pd.concat(dfs_list, axis=0, ignore_index=True)
    return concatenated_df

def process_pipeline(directory_path):
    """
    Process the data pipeline.
    """
    # Load dataframes from directory
    dfs = load_dataframes_from_directory(directory_path)
    
    cleaned_dfs = []

    # Clean each dataframe
    for subdir_name, file_dict in dfs.items():
        for file_name, df in file_dict.items():
            cleaned_df = clean_dataframe(df)
            cleaned_dfs.append(cleaned_df)

    # Concatenate all cleaned dataframes into a single dataframe
    final_df = pd.concat(cleaned_dfs, axis=0, ignore_index=True)  # Using pd.concat directly on the cleaned_dfs list

    # Save the final dataframe  
    final_df.to_csv(FINAL_DATA_DIR / 'final_dataframe.csv', index=False)
    final_df.to_excel(FINAL_DATA_DIR / 'final_dataframe.xlsx', index=False)
    return final_df


# Execute the pipeline
final_dataframe = process_pipeline(PROCESSED_DATA_DIR)


'\nopenai.api_key = os.getenv("OPENAI_API_KEY")\n\n\nload_dotenv()\n\n# Template for the prompt\nprompt_template = "{description}. Classify the description if it falls into \'Commercial\', \'Residential\', \'Industrial\', or \'Mixed Use\'. Otherwise, label it \'Other\'"\n\ndef get_response_from_langchain(description, retries=3, delay=5):\n    #Use OpenAI\'s API to fill null values and classify the description to determine the type of use.\n    for attempt in range(retries):\n        try:\n            time.sleep(3)\n            # Using OpenAI\'s API to get the response\n            messages = [{"role": "user", "content": prompt_template.format(description=description)}]\n            response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages)\n            \n            # Extract the model\'s message from the returned messages\n            answer = response[\'choices\'][0][\'message\'][\'content\'].strip()\n            \n            # Check if the response starts wit

In [5]:
final_dataframe.columns

Index(['address', 'description', 'projectName', 'typeOfUse', 'status',
       'recentUpdate', 'email', 'phone', 'city', 'planner'],
      dtype='object')

In [6]:
final_dataframe.typeOfUse.isna().sum()

247

In [24]:
df = final_dataframe 

In [ ]:
df.to_excel('example.xlsx')

In [25]:
df

,address,description,projectName,typeOfUse,status,recentUpdate,email,phone,city,planner,explicitUses
0,321 W Katella Ave,The Applicant Proposes Adjustment No. 14 To Th...,[DEV2010-00166J] ANAHEIM GARDENWALK - STC,Other,Withdrawn,12/10/2019,EThien@anaheim.net,(714) 765-4568,Anaheim,Elain Thienprasiddhi,Commercial
1,500 W Disney Way,Amend The Development Agreement Between The Ci...,[DEV2010-0166I] ANAHEIM GARDENWALK - WESTGATE,Other,In Review,2/14/2019,Planning@anaheim.net,(714) 765-5139,Anaheim,Anaheim Planning Office,Commercial
2,8163 E Kaiser Blvd,The Applicant Proposes To Amend An Existing Co...,[DEV2013-00024C] INFLUENCE CHURCH,Commercial,Approved,7/29/2020,Planning@anaheim.net,(714) 765-5139,Anaheim,Anaheim Planning Office,Major Residential Project
3,1700 S Harbor Blvd,A Request To Amend The Development Agreement B...,[DEV2015-00120A] 1700 S. HARBOR HOTEL,Other,Approved,10/2/2019,EThien@anaheim.net,(714) 765-4568,Anaheim,Elain Thienprasiddhi,Commercial
4,3111 W Orange Ave,Request To Amend A Conditional Use Permit To P...,[DEV2018-00130A] DAYBREAK UNIVERSITY,Commercial,In Review,9/15/2021,TGorham@Anaheim.net,(714) 765-4947,Anaheim,Thomas Gorham,Major Residential Project
...,...,...,...,...,...,...,...,...,...,...,...
658,1561 W. Sunflower Avenue (Ward 4),South Coast Plaza (Applicant & Property Owner)...,The Village Specific Plan,NaN,Development Project Review,"Updated: November 1st, 2023",planning@santa-ana.org,(714) 667-2732,Santa Ana,NaN,Commercial
659,601 N. Fairview Street (Ward 5),The applicant is proposing to rezone the subje...,Vista Heritage Development Project,NaN,Development Project Review,"Updated: October 11th, 2023",FArias@santa-ana.org,(714) 667-2792,Santa Ana,Fernanda Arias — Assistant Planner I,Commercial
660,2300 S. Red Hill Avenue (Ward 6),"Arrimus Capital, “Applicant”, has submitted an...",Warner Redhill Mixed-Use Development (Formerly...,NaN,Under Construction,"Updated: October 30th, 2021",JGuevara@santa-ana.org,(714) 647-5481,Santa Ana,Project Manager: Jerry C. Guevara — Senior Pla...,Mixed Use
661,2534 W. Westminster Avenue (Ward 5),"The applicant, Community Development Partners,...",Westview Housing,NaN,Under Construction,"Updated: October 30th, 2023",APezeshkpour@santa-ana.org,(714) 647-5882,Santa Ana,"Ali Pezeshkpour, AICP — Planning Manager",Commercial


In [26]:
def classify_type_of_use(description, type_of_use="Other"):
    # Check if description is not a string
    if not isinstance(description, str):
        # If not, convert to string or handle it however you see fit
        description = str(description)

    description = description.upper()
    if (
        "MIXED USE" in description
        or "MIXED_USE" in description
        or "MIXED-USE" in description
    ):
        type_of_use = "Mixed Use"
    elif (
        "ADU" in description
        or "ACCESSORY DWELLING UNIT" in description
        or "ACCESSORY" in description
    ):
        type_of_use = "Minor Residential Project"
    elif (
        "COMMERCIAL" in description
        or "STORE" in description
        or "RETAIL" in description
        or "CAR WASH" in description
        #or "INSURANCE" in description
        #or "OFFICE" in description
        or "RESTAURANT" in description
        or "RESORT" in description
        or "HOTEL" in description
        or "MOTEL" in description
    ):
        type_of_use = "Commercial"
    elif (
        "RESIDENTIAL" in description
        or "UNIT" in description
        or "FAMILY" in description
        or "APARTMENT" in description
        or "TOWNHOME" in description
        or "HOUSING" in description
        or "CDR" in description
        or "DUPLEX" in description
        or "TRANSITIONAL HOUSING" in description
        or "CHURCH" in description
        or "CONDOMINIUMS" in description
        or "CONDO" in description
    ):
        type_of_use = "Major Residential Project"
    elif (
        "INDUSTRIAL" in description
        or "WAREHOUSE" in description
        or "STORAGE" in description
        or "MANUFACTURING" in description
        or "WCF" in description
        or "FACILITY" in description
    ):
        type_of_use = "Industrial"
    elif "TPM" in description or "TENTATIVE PARCEL MAP" in description:
        type_of_use = "Tentaive Parcel Map"
    elif "VARIANCE" in description or "CONSOLIDATE" in description:
        type_of_use = "Variance Request"
    elif "DEMO" in description or "DEMOLITION" in description:
        type_of_use = "Demolition"
    elif (
        "YARD" in description
        or "FENCE" in description
        or "GARAGE" in description
        or "RESIDENCE" in description
    ):
        type_of_use = "Private Home Modification"
    elif (
        "TI" in description
        or "TENANT IMPROVEMENT" in description
        or "T.I" in description
        or 'Ti'
        or 'T.I.'
    ):
        type_of_use = "Tenant Improvement"
    elif "VTTM" in description or "VESTING TENTATIVE TRACT MAP" in description:
        type_of_use = "Vesting Tentative Tract Map"
    elif "CLASSROOM" in description or "SCHOOL" in description:
        type_of_use = "Education/Institution"
    else:
        type_of_use = "Other"

    return type_of_use

# Usage in the dataframe iteration
for index, row in df.iterrows():
    description = row["description"]
    type_of_use = classify_type_of_use(description)
    df.at[index, "explicitUses"] = type_of_use

In [13]:
df.explicitUses.unique()

array(['Commercial', 'Major Residential Project', 'Tenant Improvement',
       'Minor Residential Project', 'Demolition', 'Industrial',
       'Variance Request', 'Tentaive Parcel Map', 'Other',
       'Private Home Modification', 'Mixed Use', 'Education/Institution'],
      dtype=object)

In [31]:
df

,address,description,projectName,typeOfUse,status,recentUpdate,email,phone,city,planner,explicitUses
0,321 W Katella Ave,The Applicant Proposes Adjustment No. 14 To Th...,[DEV2010-00166J] ANAHEIM GARDENWALK - STC,Other,Withdrawn,12/10/2019,EThien@anaheim.net,(714) 765-4568,Anaheim,Elain Thienprasiddhi,Commercial
1,500 W Disney Way,Amend The Development Agreement Between The Ci...,[DEV2010-0166I] ANAHEIM GARDENWALK - WESTGATE,Other,In Review,2/14/2019,Planning@anaheim.net,(714) 765-5139,Anaheim,Anaheim Planning Office,Commercial
2,8163 E Kaiser Blvd,The Applicant Proposes To Amend An Existing Co...,[DEV2013-00024C] INFLUENCE CHURCH,Commercial,Approved,7/29/2020,Planning@anaheim.net,(714) 765-5139,Anaheim,Anaheim Planning Office,Major Residential Project
3,1700 S Harbor Blvd,A Request To Amend The Development Agreement B...,[DEV2015-00120A] 1700 S. HARBOR HOTEL,Other,Approved,10/2/2019,EThien@anaheim.net,(714) 765-4568,Anaheim,Elain Thienprasiddhi,Commercial
4,3111 W Orange Ave,Request To Amend A Conditional Use Permit To P...,[DEV2018-00130A] DAYBREAK UNIVERSITY,Commercial,In Review,9/15/2021,TGorham@Anaheim.net,(714) 765-4947,Anaheim,Thomas Gorham,Major Residential Project
...,...,...,...,...,...,...,...,...,...,...,...
658,1561 W. Sunflower Avenue (Ward 4),South Coast Plaza (Applicant & Property Owner)...,The Village Specific Plan,NaN,Development Project Review,"Updated: November 1st, 2023",planning@santa-ana.org,(714) 667-2732,Santa Ana,NaN,Commercial
659,601 N. Fairview Street (Ward 5),The applicant is proposing to rezone the subje...,Vista Heritage Development Project,NaN,Development Project Review,"Updated: October 11th, 2023",FArias@santa-ana.org,(714) 667-2792,Santa Ana,Fernanda Arias — Assistant Planner I,Industrial
660,2300 S. Red Hill Avenue (Ward 6),"Arrimus Capital, “Applicant”, has submitted an...",Warner Redhill Mixed-Use Development (Formerly...,NaN,Under Construction,"Updated: October 30th, 2021",JGuevara@santa-ana.org,(714) 647-5481,Santa Ana,Project Manager: Jerry C. Guevara — Senior Pla...,Mixed Use
661,2534 W. Westminster Avenue (Ward 5),"The applicant, Community Development Partners,...",Westview Housing,NaN,Under Construction,"Updated: October 30th, 2023",APezeshkpour@santa-ana.org,(714) 647-5882,Santa Ana,"Ali Pezeshkpour, AICP — Planning Manager",Major Residential Project


In [32]:
df.dropna(subset=['description'], inplace=True)
df.shape

(645, 11)

## Use OPENAI API to infer insights from the text

In [ ]:
prompt_template1 = "{description}. From the details, determine the following: \
\n1. What is the type of development action? (Options: 'Amendment Request', 'Construction', 'Permit Application', 'Review & Redesignation') \
\n2. What type of facility is being discussed? (Options: 'Residential', 'Commercial', 'Recreational') Also, list any specific features or additions associated with it. \
\n3. Provide any size and location-specific details mentioned in the description, like square footage, number of units, and if it's part of a bigger development area or adjacent to certain landmarks."

prompt_template2 = "Given the description: {description}, answer the following: \
\n- What is the exact development action being taken? \
\n- Which category (Residential, Commercial, Recreational, etc.) best describes the facility being discussed? Are there any notable features or additions mentioned for this facility? \
\n- Can you identify any specifics regarding the size and location of the property or project, such as square footage, number of rooms, and its relation to other structures or landmarks?"

prompt_template3 = "Imagine you are analyzing the description: {description} to create a detailed report. Please classify and specify the following details: \
\na) The development action being undertaken. \
\nb) The main type of facility being highlighted and its distinct features or additions. \
\nc) Any information related to the size of the property or project and its location characteristics."


In [33]:
from dotenv import load_dotenv
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")


load_dotenv()

# Template for the prompt
prompt_template = "{description}. Based on the information provided, determine the primary use of the described facility or project. Possible categories are 'Commercial', 'Residential', 'Industrial', and 'Mixed Use'. If the description doesn't fit into any of these categories, classify it as 'Other'. Provide a brief reasoning for the chosen category."

def get_response_from_langchain(description, retries=3, delay=5):
    #Use OpenAI's API to fill null values and classify the description to determine the type of use.
    for attempt in range(retries):
        try:
            time.sleep(6)
            # Using OpenAI's API to get the response
            messages = [{"role": "user", "content": prompt_template.format(description=description)}]
            response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages)
            
            # Extract the model's message from the returned messages
            answer = response['choices'][0]['message']['content'].strip()
            
            # Check if the response starts with "Other" and then extract the two-word description
            if answer.startswith("Other"):
                return answer
            
            # If the response doesn't start with "Other", return just the category
            return answer.split()[0]
        
        except Exception as e:
            if attempt < retries - 1:
                print(f"Error: {e}. Retrying in {delay} seconds...")
                time.sleep(delay)  # Wait for a specified delay before retrying
            else:
                print(f"Error: {e}. Failed after {retries} attempts.")
                return "Error"

    return None

# Apply only to rows where 'typeOfUse' is NaN
#mask = final_dataframe['typeOfUse'].isna()
df['specifiedUse'] = df['description'].apply(get_response_from_langchain)



Error: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600). Retrying in 5 seconds...
Error: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600). Retrying in 5 seconds...
Error: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')). Retrying in 5 seconds...


In [27]:
import plotly.express as px

# Count the instances for each unique value in 'explicitUses'
value_counts = df['explicitUses'].value_counts().reset_index()
value_counts.columns = ['Type of Use', 'Number of Instances']

# Create the bar chart
fig = px.bar(value_counts,
             x='Type of Use', 
             y='Number of Instances',
             title='Distribution of Explicit Uses')

fig.show()


In [ ]:
fig = px.histogram(df, x='status', title="Distribution of Project Statuses")
fig.show()


In [37]:
fig = px.histogram(df, x='city', color='typeOfUse', title="Distribution of Development Types by City")
fig.show()


In [36]:
approval_rates = df.groupby('typeOfUse').apply(lambda group: len(group[group['status'] == 'Approved']) / len(group)).reset_index()
approval_rates.columns = ['typeOfUse', 'approval_rate']

fig = px.bar(approval_rates, x='typeOfUse', y='approval_rate', title="Approval Rate by Project Type")
fig.show()

In [35]:
df['specifiedUse'].value_counts()

specifiedUse
The                                                                                                                                                                                                                                                                                                                                                                                                                                                                              276
Based                                                                                                                                                                                                                                                                                                                                                                                                                                                                            272
Residential.                                     

In [ ]:
df['days_in_review'] = (pd.to_datetime(df['recentUpdate']) - pd.to_datetime(df['submissionDate'])).dt.days
in_review_df = df[df['status'] == 'In Review']

fig = px.histogram(in_review_df, x='days_in_review', title="Duration of Projects 'In Review'")
fig.show()

In [29]:
desc_explicit_uses = df[['projectName', 'description', 'explicitUses']]

In [30]:
desc_explicit_uses.to_excel('desc_explicit_uses.xlsx')

In [12]:
for index, row in df.iterrows():
    description = row["description"]
    if not isinstance(description, str):
        print(f"Non-string description at index {index}: {description}")
    type_of_use = classify_type_of_use(description)
    df.at[index, "explicitUses"] = type_of_use


Non-string description at index 435: nan


AttributeError: 'float' object has no attribute 'upper'